<a href="https://colab.research.google.com/github/DangPhan23/Final_Exams/blob/main/CUOI_KY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Khai báo các thư viện
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score


In [2]:
# Truy xuất hình ảnh và nhãn của ảnh
image_path=r"/content/drive/MyDrive/AI/dulieu/Train"
data = []
labels = []
classes = 43
for i in range(classes):
    path = os.path.join(image_path, str(i)) #0-42
    images = os.listdir(path)
    for img in images:
        try:
          image = Image.open(path +'/'+ img)
          image = image.resize((30,30))
          image = np.array(image)
          data.append(image)
          labels.append(i)
        except:
          print("Error loading image")
#Chuyển đổi danh sách thành mảng có nhiều mảng
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape) 

(39253, 30, 30, 3) (39253,)


In [3]:
#Chia tập dữ liệu huấn luyện và thử nghiệm
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print("training shape: ",x_train.shape, y_train.shape)
print("testing shape: ",x_test.shape, y_test.shape)

training shape:  (31402, 30, 30, 3) (31402,)
testing shape:  (7851, 30, 30, 3) (7851,)


In [4]:
#Chuyển đổi nhãn thành một mã
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [5]:
#Xây dựng mô hình
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                    

In [6]:
#Đào tạo mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
epochs = 15
history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/15
491/491 [==============================] - 16s 9ms/step - loss: 2.5789 - accuracy: 0.4380 - val_loss: 0.4607 - val_accuracy: 0.8884
Epoch 2/15
491/491 [==============================] - 4s 8ms/step - loss: 0.5634 - accuracy: 0.8374 - val_loss: 0.1487 - val_accuracy: 0.9694
Epoch 3/15
491/491 [==============================] - 4s 8ms/step - loss: 0.3005 - accuracy: 0.9165 - val_loss: 0.0598 - val_accuracy: 0.9836
Epoch 4/15
491/491 [==============================] - 3s 7ms/step - loss: 0.2100 - accuracy: 0.9433 - val_loss: 0.0444 - val_accuracy: 0.9883
Epoch 5/15
491/491 [==============================] - 4s 8ms/step - loss: 0.1758 - accuracy: 0.9518 - val_loss: 0.0414 - val_accuracy: 0.9901
Epoch 6/15
491/491 [==============================] - 4s 8ms/step - loss: 0.1646 - accuracy: 0.9547 - val_loss: 0.0558 - val_accuracy: 0.9864
Epoch 7/15
491/491 [==============================] - 4s 8ms/step - loss: 0.1510 - accuracy: 0.9598 - val_loss: 0.0317 - val_accuracy: 0.9910
Epoch

In [7]:
#testing accuracy on test dataset
y_test = pd.read_csv('/content/drive/MyDrive/AI/dulieu/Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
   image = Image.open(img)
   image = image.resize((30,30))
   data.append(np.array(image))
X_test=np.array(data)
predict_x = model.predict(X_test) 
pred = np.argmax(predict_x,axis=1)
#Accuracy with the test data
print("Độ chính xác của mô hình: ",accuracy_score(labels, pred)*100)

7/7 [==============================] - 0s 17ms/step
Độ chính xác của mô hình:  96.4824120603015


In [ ]:

model.save('/content/drive/MyDrive/AI/dulieu/traffic_classifier.h5')#to save